# 01 - AML & MLflow 接続セットアップ

このノートブックでは、Azure Machine Learning Workspace への接続と、MLflow トラッキング URI の設定を行います。

In [ ]:
from azureml.core import Workspace
import mlflow

In [ ]:
# .azureml/config.json を用いて Workspace に接続
ws = Workspace.from_config(path=".azureml/config.json")
print(f"Workspace name: {ws.name}")

In [ ]:
# MLflow に AML ワークスペースのトラッキング URI を指定
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

In [ ]:
# 実験の作成（存在しない場合）
experiment_name = "pointnet-semseg"
mlflow.set_experiment(experiment_name)
print(f"Experiment set: {experiment_name}")

## 🔐 Entra ID アプリケーション登録

AML ワークスペースにプログラムからアクセスするために、
1) Entra ID でアプリケーション (Service Principal) を登録し、
2) 適切なロール (例: Contributor) を付与します。

以下の Azure CLI コマンドを実行してください。

In [ ]:
# Azure CLI にログインしていない場合は、以下のコマンドを実行してください
!az login

In [ ]:
# Service Principal の作成
# <subscription-id> と <rg-name> は Azure Portal で確認してください。
# <workspace-name> は Azure Machine Learning ワークスペースの名前です。
!az ad sp create-for-rbac \
    --name "pointnet-app" \
    --role Contributor \
    --scopes "/subscriptions/<subscription-id>/resourceGroups/<rg-name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace-name>"

# 出力された JSON (clientId, clientSecret, tenantId, subscriptionId) を.env ファイルに保存します。

サービスプリンシパルが作成されたら、Azure Machine Learning ワークスペースに対し、 **AzureML データ科学者** ロールを割り当てる。

### 環境変数の設定例
```bash
export AZURE_SUBSCRIPTION_ID=<subscription-id>
export AZURE_RESOURCE_GROUP=<rg-name>
export AZURE_WORKSPACE_NAME=<workspace-name>
export AZURE_TENANT_ID=<tenantId-from-json>
export AZURE_CLIENT_ID=<clientId-from-json>
export AZURE_CLIENT_SECRET=<clientSecret-from-json>
```

### Azure Machine Learning への接続方法

```python
from azure.ai.ml import MLClient
from azure.identity import ClientSecretCredential


credential = ClientSecretCredential(
    tenant_id=os.environ['AZURE_TENANT_ID'],
    client_id=os.environ['AZURE_CLIENT_ID'],
    client_secret=os.environ['AZURE_CLIENT_SECRET']
)

ml_client = MLClient(
    credential,
    os.environ['AZURE_SUBSCRIPTION_ID'],
    os.environ['AZURE_RESOURCE_GROUP'],
    os.environ['AZURE_WORKSPACE_NAME']
)